In [1]:
import os
import sys
sys.path.append("../cnn")
sys.path.append("../utils")

import torch
from torch.autograd import Variable

from ipywidgets import interact, fixed
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# import seaborn as sns; sns.set()

from cnn_md import CNNMultidecoder, CNNVariationalMultidecoder
from cnn_md import CNNDomainAdversarialMultidecoder
from cnn_md import CNNGANMultidecoder
from hao_data import HaoEvalDataset

In [5]:
# Set up environment variables for the model we want to examine
# NOT necessarily the current environment variables!!

feat_dim=80
left_context=5
right_context=5

optimizer="Adam"
# optimizer="SGD"
learning_rate="0.0001" # Use string instead of float to prevent Python's auto-formatting...
# learning_rate="0.000001" # Use string instead of float to prevent Python's auto-formatting...
epochs=25
# batch_size=256
batch_size=128

enc_channels=[256, 256]
enc_kernels=[3, 3]
enc_downsamples=[3, 3]
enc_fc=[]

latent_dim=256

dec_fc=[]
dec_channels=[256, 256]
dec_kernels=[3, 3]
dec_upsamples=[3, 3]

decoder_classes=["ihm", "sdm1"]

use_batch_norm=False
use_batch_norm_str = "true" if use_batch_norm else "false"
activation="ReLU"
weight_init="xavier_uniform"

enc_channels_delim="_" + "_".join(map(str, enc_channels))
if len(enc_channels) == 0:
     # need to recreate bash join behavior with empty array
    enc_channels_delim="_"
enc_kernels_delim="_" + "_".join(map(str, enc_kernels))
if len(enc_kernels) == 0:
     # need to recreate bash join behavior with empty array
    enc_kernels_delim="_"
enc_downsamples_delim="_" + "_".join(map(str, enc_downsamples))
if len(enc_downsamples) == 0:
     # need to recreate bash join behavior with empty array
    enc_downsamples_delim="_"
enc_fc_delim="_" + "_".join(map(str, enc_fc))
if len(enc_fc) == 0:
     # need to recreate bash join behavior with empty array
    enc_fc_delim="_"
    
dec_fc_delim="_" + "_".join(map(str, dec_fc))
if len(dec_fc) == 0:
     # need to recreate bash join behavior with empty array
    dec_fc_delim="_"
dec_channels_delim="_" + "_".join(map(str, dec_channels))
if len(dec_channels) == 0:
     # need to recreate bash join behavior with empty array
    dec_channels_delim="_"
dec_kernels_delim="_" + "_".join(map(str, dec_kernels))
if len(dec_kernels) == 0:
     # need to recreate bash join behavior with empty array
    dec_kernels_delim="_"
dec_upsamples_delim="_" + "_".join(map(str, dec_upsamples))
if len(dec_upsamples) == 0:
     # need to recreate bash join behavior with empty array
    dec_upsamples_delim="_"

debug_model = False
debug_str = "true" if debug_model else "false"

'''
expt_name = "ENC_C%s_K%s_P%s_F%s/LATENT_%d/DEC_F%s_C%s_K%s_P%s/ACT_%s_BN_%s_WEIGHT_INIT_%s/OPT_%s_LR_%s_EPOCHS_%d_BATCH_%d_DEBUG_%s" % (enc_channels_delim,
                                                                                                                                        enc_kernels_delim, 
                                                                                                                                        enc_downsamples_delim, 
                                                                                                                                        enc_fc_delim,
                                                                                                                                        latent_dim,
                                                                                                                                        dec_fc_delim,
                                                                                                                                        dec_channels_delim,
                                                                                                                                        dec_kernels_delim,                                                                                                                   dec_upsamples_delim,
                                                                                                                                        activation,
                                                                                                                                        use_batch_norm_str,
                                                                                                                                        weight_init,
                                                                                                                                        optimizer,
                                                                                                                                        learning_rate,
                                                                                                                                        epochs,
                                                                                                                                        batch_size,
                                                                                                                                        debug_str)



'''

use_backtranslation = True
use_backtranslation_str = "true" if use_backtranslation else "false"

strided = False
strided_str = "true" if strided else "false"

expt_name = "STRIDED_%s_BACKTRANS_%s_ENC_C%s_K%s_P%s_F%s/LATENT_%d/DEC_F%s_C%s_K%s_P%s/ACT_%s_BN_%s_WEIGHT_INIT_%s/OPT_%s_LR_%s_EPOCHS_%d_BATCH_%d_DEBUG_%s" % (strided_str,
                                                                                                                                        use_backtranslation_str,
                                                                                                                                        enc_channels_delim,
                                                                                                                                        enc_kernels_delim, 
                                                                                                                                        enc_downsamples_delim, 
                                                                                                                                        enc_fc_delim,
                                                                                                                                        latent_dim,
                                                                                                                                        dec_fc_delim,
                                                                                                                                        dec_channels_delim,
                                                                                                                                        dec_kernels_delim,                                                                                                                   dec_upsamples_delim,
                                                                                                                                        activation,
                                                                                                                                        use_batch_norm_str,
                                                                                                                                        weight_init,
                                                                                                                                        optimizer,
                                                                                                                                        learning_rate,
                                                                                                                                        epochs,
                                                                                                                                        batch_size,
                                                                                                                                        debug_str)





time_dim = (left_context + right_context + 1)
freq_dim = feat_dim

noise_ratio=0.0
print("Noise ratio %s" % str(noise_ratio))

print("Using experiment %s" % expt_name)

# dataset = "ami-0.1"
dataset = "ami-full"

domain_adv_fc = [512, 512]
domain_adv_fc_delim="_" + "_".join(map(str, domain_adv_fc))
if len(domain_adv_fc) == 0:
     # need to recreate bash join behavior with empty array
    domain_adv_fc_delim="_"
domain_adv_activation = "LeakyReLU"

gan_fc = [512, 512]
gan_fc_delim="_" + "_".join(map(str, gan_fc))
if len(gan_fc) == 0:
     # need to recreate bash join behavior with empty array
    gan_fc_delim="_"
gan_activation = "LeakyReLU"

Noise ratio 0.0
Using experiment STRIDED_false_BACKTRANS_true_ENC_C_256_256_K_3_3_P_3_3_F_/LATENT_256/DEC_F__C_256_256_K_3_3_P_3_3/ACT_ReLU_BN_false_WEIGHT_INIT_xavier_uniform/OPT_Adam_LR_0.0001_EPOCHS_25_BATCH_128_DEBUG_false


In [6]:
# Set up datasets for IHM, SDM1 baselines (dev set only)
dataset_name = "ami-0.1"
current_feats = "/data/sls/r/u/atitus5/meng/%s" % dataset_name
ihm_baseline = HaoEvalDataset(os.path.join(current_feats, "ihm-dev-norm.blogmel.scp"))
sdm1_baseline = HaoEvalDataset(os.path.join(current_feats, "sdm1-dev-norm.blogmel.scp"))
print("Set up baseline dev datasets")

Set up baseline dev datasets


## IF DATA AUGMENTATION NOT RUN YET

In [7]:
model_type = "ae"
domain_adversarial = True
gan = False
checkpoint_dir = "/data/sls/scratch/atitus5/meng/models/cnn/%s/%s" % (dataset,
                                                                      expt_name)

if domain_adversarial:
    checkpoint_file = "%s/best_cnn_domain_adversarial_fc_%s_act_%s_%s_ratio%s_md.pth.tar" % (checkpoint_dir,
                                                                      domain_adv_fc_delim,
                                                                      domain_adv_activation,
                                                                      model_type,
                                                                      str(noise_ratio))
    
    if model_type == "ae":
        model = CNNDomainAdversarialMultidecoder(freq_dim=freq_dim,
                                splicing=[left_context, right_context], 
                                enc_channel_sizes=enc_channels,
                                enc_kernel_sizes=enc_kernels,
                                enc_downsample_sizes=enc_downsamples,
                                enc_fc_sizes=enc_fc,
                                latent_dim=latent_dim,
                                dec_fc_sizes=dec_fc,
                                dec_channel_sizes=dec_channels,
                                dec_kernel_sizes=dec_kernels,
                                dec_upsample_sizes=dec_upsamples,
                                activation=activation,
                                decoder_classes=decoder_classes,
                                use_batch_norm=use_batch_norm,
                                weight_init=weight_init,
                                domain_adv_fc_sizes=domain_adv_fc,
                                domain_adv_activation=domain_adv_activation)
    elif model_type == "vae":
        print("Adversarial VAE not supported yet")
elif gan:
    checkpoint_file = "%s/best_cnn_gan_fc_%s_act_%s_%s_ratio%s_md.pth.tar" % (checkpoint_dir,
                                                                      gan_fc_delim,
                                                                      gan_activation,
                                                                      model_type,
                                                                      str(noise_ratio))
    
    if model_type == "ae":
        model = CNNGANMultidecoder(freq_dim=freq_dim,
                                splicing=[left_context, right_context], 
                                enc_channel_sizes=enc_channels,
                                enc_kernel_sizes=enc_kernels,
                                enc_downsample_sizes=enc_downsamples,
                                enc_fc_sizes=enc_fc,
                                latent_dim=latent_dim,
                                dec_fc_sizes=dec_fc,
                                dec_channel_sizes=dec_channels,
                                dec_kernel_sizes=dec_kernels,
                                dec_upsample_sizes=dec_upsamples,
                                activation=activation,
                                decoder_classes=decoder_classes,
                                use_batch_norm=use_batch_norm,
                                weight_init=weight_init,
                                gan_fc_sizes=gan_fc,
                                gan_activation=gan_activation)
    elif model_type == "vae":
        print("Generative adversarial VAE not supported yet")
else:
    checkpoint_file = "%s/best_cnn_%s_ratio%s_md.pth.tar" % (checkpoint_dir,
                                                             model_type,
                                                             str(noise_ratio))
    
    if model_type == "ae":
        model = CNNMultidecoder(freq_dim=freq_dim,
                                splicing=[left_context, right_context], 
                                enc_channel_sizes=enc_channels,
                                enc_kernel_sizes=enc_kernels,
                                enc_downsample_sizes=enc_downsamples,
                                enc_fc_sizes=enc_fc,
                                latent_dim=latent_dim,
                                dec_fc_sizes=dec_fc,
                                dec_channel_sizes=dec_channels,
                                dec_kernel_sizes=dec_kernels,
                                dec_upsample_sizes=dec_upsamples,
                                activation=activation,
                                decoder_classes=decoder_classes,
                                use_batch_norm=use_batch_norm,
                                weight_init=weight_init)
    elif model_type == "vae":
        model = CNNVariationalMultidecoder(freq_dim=freq_dim,
                                splicing=[left_context, right_context], 
                                enc_channel_sizes=enc_channels,
                                enc_kernel_sizes=enc_kernels,
                                enc_downsample_sizes=enc_downsamples,
                                enc_fc_sizes=enc_fc,
                                latent_dim=latent_dim,
                                dec_fc_sizes=dec_fc,
                                dec_channel_sizes=dec_channels,
                                dec_kernel_sizes=dec_kernels,
                                dec_upsample_sizes=dec_upsamples,
                                activation=activation,
                                decoder_classes=decoder_classes,
                                use_batch_norm=use_batch_norm,
                                weight_init=weight_init)
        
        
checkpoint = torch.load(checkpoint_file, map_location=lambda storage,loc: storage)
model.load_state_dict(checkpoint["state_dict"])
model.eval()

CNNDomainAdversarialMultidecoder(
  (encoder_conv): Sequential(
    (conv2d_0): Conv2d (1, 256, kernel_size=(3, 3), stride=(1, 1))
    (ReLU_0): ReLU()
    (maxpool2d_0): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), dilation=(1, 1))
    (conv2d_1): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1))
    (ReLU_1): ReLU()
    (maxpool2d_1): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), dilation=(1, 1))
  )
  (encoder_fc): Sequential(
    (lin_final): Linear(in_features=14336, out_features=256)
  )
  (decoder_fc_ihm): Sequential(
    (ReLU_final): ReLU()
    (lin_final): Linear(in_features=256, out_features=14336)
  )
  (decoder_deconv_ihm): Sequential(
    (ReLU_0): ReLU()
    (maxunpool2d_0): MaxUnpool2d(kernel_size=(1, 3), stride=(1, 3), padding=(0, 0))
    (conv2d_0): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (ReLU_1): ReLU()
    (maxunpool2d_1): MaxUnpool2d(kernel_size=(1, 3), stride=(1, 3), padding=(0, 0))
    (conv2d_1): Conv2d (256, 1, kernel_size=(3

In [8]:
def augmentFeats(model, feats, decoder_class):
    feats_numpy = feats.reshape((-1, freq_dim))
    num_frames = feats_numpy.shape[0]
    decoded_feats = np.empty((num_frames, freq_dim))
    for i in range(num_frames):
        frame_spliced = np.zeros((time_dim, freq_dim))
        frame_spliced[left_context - min(i, left_context):left_context, :] = feats_numpy[i - min(i, left_context):i, :]
        frame_spliced[left_context, :] = feats_numpy[i, :]
        frame_spliced[left_context + 1:left_context + 1 + min(num_frames - i - 1, right_context), :] = feats_numpy[i + 1:i + 1 + min(num_frames - i - 1, right_context), :]
        frame_tensor = Variable(torch.FloatTensor(frame_spliced))

        if model_type == "ae":
            recon_frames = model.forward_decoder(frame_tensor, decoder_class)
        elif model_type == "vae":
            recon_frames, mu, logvar = model.forward_decoder(frame_tensor, decoder_class)
        else:
            print("Unrecognized model type %s" % model_type)
        recon_frames_numpy = recon_frames.cpu().data.numpy().reshape((-1, freq_dim))
        decoded_feats[i, :] = recon_frames_numpy[left_context:left_context + 1, :]
    return decoded_feats

In [9]:
# color_map = "coolwarm"
color_map = "viridis"
def plotParallelUtts(utt_id_idx):
    fig, axarr = plt.subplots(10, sharex=True)
    
    fig.set_size_inches(8.5, 22)
    
    # IHM baseline
    ihm_baseline_utt_id = ihm_baseline.utt_ids[utt_id_idx]
    ihm_baseline_feats = ihm_baseline.feats_for_uttid(ihm_baseline_utt_id)
    axarr[0].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[0].imshow(np.transpose(ihm_baseline_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[0].set_title("IHM")
    
    # SDM1 baseline
    sdm1_baseline_utt_id = sdm1_baseline.utt_ids[utt_id_idx]
    sdm1_baseline_feats = sdm1_baseline.feats_for_uttid(sdm1_baseline_utt_id)
    axarr[1].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[1].imshow(np.transpose(sdm1_baseline_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[1].set_title("SDM1")
    
    # IHM->IHM
    ihm_ihm_feats = augmentFeats(model, ihm_baseline_feats, "ihm")
    axarr[2].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[2].imshow(np.transpose(ihm_ihm_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[2].set_title("IHM>IHM")
    
    # IHM->SDM1
    ihm_sdm1_feats = augmentFeats(model, ihm_baseline_feats, "sdm1")
    axarr[3].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[3].imshow(np.transpose(ihm_sdm1_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[3].set_title("IHM>SDM1")
    
    # IHM transform difference
    ihm_transform_diff = ihm_sdm1_feats - ihm_ihm_feats
    axarr[4].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[4].imshow(np.transpose(ihm_transform_diff), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[4].set_title("IHM>SDM1 and IHM>IHM difference")
    
    # SDM1->SDM1
    sdm1_sdm1_feats = augmentFeats(model, sdm1_baseline_feats, "sdm1")
    axarr[5].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[5].imshow(np.transpose(sdm1_sdm1_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[5].set_title("SDM1>SDM1")
    
    # SDM1->IHM
    sdm1_ihm_feats = augmentFeats(model, sdm1_baseline_feats, "ihm")
    axarr[6].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[6].imshow(np.transpose(sdm1_ihm_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[6].set_title("SDM1>IHM")
    
    # SDM1 transform difference
    sdm1_transform_diff = sdm1_ihm_feats - sdm1_sdm1_feats
    axarr[7].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[7].imshow(np.transpose(sdm1_transform_diff), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[7].set_title("SDM1>IHM and SDM1>SDM1 difference")
    
    # IHM backtranslated
    ihm_backtrans_feats = augmentFeats(model, ihm_sdm1_feats, "ihm")
    axarr[8].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[8].imshow(np.transpose(ihm_backtrans_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[8].set_title("IHM backtranslated")
    
    # SDM1 backtranslated
    sdm1_backtrans_feats = augmentFeats(model, sdm1_ihm_feats, "sdm1")
    axarr[9].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[9].imshow(np.transpose(sdm1_backtrans_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[9].set_title("SDM1 backtranslated")
    
    plt.tight_layout()
    
    # fig.savefig("%s_idx%d.eps" % (ihm_baseline_utt_id, utt_id_idx))
    
    plt.show()

# 43, 251 are good utterances (long + variations between IHM and SDM1)
interact(plotParallelUtts, utt_id_idx=range(len(ihm_baseline)))

A Jupyter Widget

<function __main__.plotParallelUtts>

In [8]:
# color_map = "coolwarm"
color_map = "viridis"
def plotIHMToSDM1(utt_id_idx):
    fig = plt.figure()
    
    fig.set_size_inches(8.5, 8.5)
    ax = plt.gca()
    
    # IHM baseline feats
    ihm_baseline_utt_id = ihm_baseline.utt_ids[utt_id_idx]
    ihm_baseline_feats = ihm_baseline.feats_for_uttid(ihm_baseline_utt_id)
    
    # IHM->SDM1
    ihm_sdm1_feats = augmentFeats(model, ihm_baseline_feats, "sdm1")
    ax.axis('off')    # Pretty-up the resulting output by removing gridlines
    ax.imshow(np.transpose(ihm_sdm1_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    ax.set_title("IHM>SDM1")
    
    # fig.savefig("%s_idx%d_IHMToSDM1.eps" % (ihm_baseline_utt_id, utt_id_idx))
    
    plt.show()

# 43, 251 are good utterances (long + variations between IHM and SDM1)
interact(plotIHMToSDM1, utt_id_idx=range(len(ihm_baseline)))

A Jupyter Widget

<function __main__.plotIHMToSDM1>

In [11]:
def realOrFake(model, feats, decoder_class):
    feats_numpy = feats.reshape((-1, freq_dim))
    num_frames = feats_numpy.shape[0]
    real_confidence = np.empty((1, num_frames))
    for i in range(num_frames):
        frame_spliced = np.zeros((time_dim, freq_dim))
        frame_spliced[left_context - min(i, left_context):left_context, :] = feats_numpy[i - min(i, left_context):i, :]
        frame_spliced[left_context, :] = feats_numpy[i, :]
        frame_spliced[left_context + 1:left_context + 1 + min(num_frames - i - 1, right_context), :] = feats_numpy[i + 1:i + 1 + min(num_frames - i - 1, right_context), :]
        frame_tensor = Variable(torch.FloatTensor(frame_spliced))

        confidence = model.forward_gan(frame_tensor, decoder_class)
        confidence_numpy = confidence.cpu().data.numpy().reshape((-1))[0]
        real_confidence[0, i] = confidence_numpy
    return real_confidence

In [12]:
def classifications(confidences):
    classifications_bool = (confidences >= 0.5)
    return classifications_bool.astype(int)

In [13]:
yesNoColorMap = "RdYlGn"
def plotRealOrFake(utt_id_idx):
    fig, axarr = plt.subplots(6, sharex=True)
    
    fig.set_size_inches(8.5, 11)
    
    # IHM baseline
    ihm_baseline_utt_id = ihm_baseline.utt_ids[utt_id_idx]
    ihm_baseline_feats = ihm_baseline.feats_for_uttid(ihm_baseline_utt_id)
    axarr[0].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[0].imshow(classifications(realOrFake(model, ihm_baseline_feats, "ihm")), origin='lower', cmap=yesNoColorMap, aspect='auto', interpolation='none', vmin=0.0, vmax=1.0)
    axarr[0].set_title("IHM")
    
    # SDM1 baseline
    sdm1_baseline_utt_id = sdm1_baseline.utt_ids[utt_id_idx]
    sdm1_baseline_feats = sdm1_baseline.feats_for_uttid(sdm1_baseline_utt_id)
    axarr[1].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[1].imshow(classifications(realOrFake(model, sdm1_baseline_feats, "sdm1")), origin='lower', cmap=yesNoColorMap, aspect='auto', interpolation='none', vmin=0.0, vmax=1.0)
    axarr[1].set_title("SDM1")
    
    # IHM->IHM
    ihm_ihm_feats = augmentFeats(model, ihm_baseline_feats, "ihm")
    axarr[2].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[2].imshow(classifications(realOrFake(model, ihm_ihm_feats, "ihm")), origin='lower', cmap=yesNoColorMap, aspect='auto', interpolation='none', vmin=0.0, vmax=1.0)
    axarr[2].set_title("IHM>IHM")
    
    # IHM->SDM1
    ihm_sdm1_feats = augmentFeats(model, ihm_baseline_feats, "sdm1")
    axarr[3].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[3].imshow(classifications(realOrFake(model, ihm_sdm1_feats, "sdm1")), origin='lower', cmap=yesNoColorMap, aspect='auto', interpolation='none', vmin=0.0, vmax=1.0)
    axarr[3].set_title("IHM>SDM1")
    
    # SDM1->SDM1
    sdm1_sdm1_feats = augmentFeats(model, sdm1_baseline_feats, "sdm1")
    axarr[4].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[4].imshow(classifications(realOrFake(model, sdm1_sdm1_feats, "sdm1")), origin='lower', cmap=yesNoColorMap, aspect='auto', interpolation='none', vmin=0.0, vmax=1.0)
    axarr[4].set_title("SDM1>SDM1")
    
    # SDM1->IHM
    sdm1_ihm_feats = augmentFeats(model, sdm1_baseline_feats, "ihm")
    axarr[5].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[5].imshow(classifications(realOrFake(model, sdm1_ihm_feats, "ihm")), origin='lower', cmap=yesNoColorMap, aspect='auto', interpolation='none', vmin=0.0, vmax=1.0)
    axarr[5].set_title("SDM1>IHM")
    
    plt.tight_layout()
    plt.show()

# 43, 251 are good utterances (long + variations between IHM and SDM1)
interact(plotRealOrFake, utt_id_idx=range(len(ihm_baseline)))

A Jupyter Widget

<function __main__.plotRealOrFake>

## IF DATA AUGMENTATION ALREADY RAN

In [9]:
# Set up augmented datasets for IHM->IHM, IHM->SDM1, SDM1->SDM1, SDM1->IHM (dev set only)
model_type = "ae"
adversarial = True
gan=False
augmented_data_base_dir = "/data/sls/scratch/atitus5/meng/augmented_data/cnn/%s/%s" % (dataset,
                                                                                       expt_name)

if adversarial:
    augmented_data_dir = "%s/adversarial_fc_%s_act_%s_%s_ratio%s" % (augmented_data_base_dir,
                                                                     adv_fc_delim,
                                                                     adv_activation,
                                                                     model_type,
                                                                     str(noise_ratio))
elif gan:
    augmented_data_dir = "%s/gan_fc_%s_act_%s_%s_ratio%s" % (augmented_data_base_dir,
                                                                     gan_fc_delim,
                                                                     gan_activation,
                                                                     model_type,
                                                                     str(noise_ratio))
else:
    augmented_data_dir = "%s/%s_ratio%s" % (augmented_data_base_dir,
                                            model_type,
                                            str(noise_ratio))

ihm_ihm = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_ihm-tar_ihm.scp"))
ihm_sdm1 = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_ihm-tar_sdm1.scp"))
sdm1_sdm1 = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_sdm1-tar_sdm1.scp"))
sdm1_ihm = HaoEvalDataset(os.path.join(augmented_data_dir, "dev-src_sdm1-tar_ihm.scp"))
print("Set up augmented dev datasets")

Set up augmented dev datasets


In [10]:
# color_map = "coolwarm"
color_map = "viridis"
def plotParallelUtts(utt_id_idx):
    fig, axarr = plt.subplots(6, sharex=True)
    
    fig.set_size_inches(8.5, 11)
    
    # IHM baseline
    ihm_baseline_utt_id = ihm_baseline.utt_ids[utt_id_idx]
    ihm_baseline_feats = np.transpose(ihm_baseline.feats_for_uttid(ihm_baseline_utt_id))
    axarr[0].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[0].imshow(ihm_baseline_feats, origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[0].set_title("IHM")
    
    # SDM1 baseline
    sdm1_baseline_utt_id = sdm1_baseline.utt_ids[utt_id_idx]
    sdm1_baseline_feats = np.transpose(sdm1_baseline.feats_for_uttid(sdm1_baseline_utt_id))
    axarr[1].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[1].imshow(sdm1_baseline_feats, origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[1].set_title("SDM1")
    
    # IHM->IHM baseline
    ihm_ihm_utt_id = ihm_ihm.utt_ids[utt_id_idx]
    ihm_ihm_feats = np.transpose(ihm_ihm.feats_for_uttid(ihm_ihm_utt_id))
    axarr[2].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[2].imshow(ihm_ihm_feats, origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[2].set_title("IHM>IHM")
    
    # IHM->SDM1 baseline
    ihm_sdm1_utt_id = ihm_sdm1.utt_ids[utt_id_idx]
    ihm_sdm1_feats = np.transpose(ihm_sdm1.feats_for_uttid(ihm_sdm1_utt_id))
    axarr[3].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[3].imshow(ihm_sdm1_feats, origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[3].set_title("IHM>SDM1")
    
    # SDM1->SDM1 baseline
    sdm1_sdm1_utt_id = sdm1_sdm1.utt_ids[utt_id_idx]
    sdm1_sdm1_feats = np.transpose(sdm1_sdm1.feats_for_uttid(sdm1_sdm1_utt_id))
    axarr[4].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[4].imshow(sdm1_sdm1_feats, origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[4].set_title("SDM1>SDM1")
    
    # SDM1->IHM baseline
    sdm1_ihm_utt_id = sdm1_ihm.utt_ids[utt_id_idx]
    sdm1_ihm_feats = np.transpose(sdm1_ihm.feats_for_uttid(sdm1_ihm_utt_id))
    axarr[5].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[5].imshow(sdm1_ihm_feats, origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[5].set_title("SDM1>IHM")
    
    plt.tight_layout()
    
    # fig.savefig("%s_idx%d.eps" % (ihm_baseline_utt_id, utt_id_idx))
    
    plt.show()

# 43, 251 are good utterances (long + variations between IHM and SDM1)
interact(plotParallelUtts, utt_id_idx=range(len(ihm_baseline)))

A Jupyter Widget

<function __main__.plotParallelUtts>